In [12]:
from pymongo import MongoClient
import pandas as pd

In [13]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [51]:
db, star = connectCollection('datlab','starbucks')
db, sch = connectCollection('datlab', 'schools')
db, air = connectCollection('datlab', 'airports')
db, night = connectCollection('datlab', 'nightclubs')
db, veg = connectCollection('datlab', 'vegan_restaurants')
db, kinder = connectCollection('datlab', 'kindergarten')

In [15]:
starque = list(star.find())

In [80]:
def geoQuery(collection,coordinates,maxdistance,mindistance):
    lista = collection.find(
        {"location":
         {"$near":
          {"$geometry":
           {"type":"Point",
            "coordinates":coordinates
           },
           "$maxDistance":maxdistance,
            "$minDistance":mindistance
          }
         }
        }
    )
    return len(list(lista))


In [127]:
db, coll = connectCollection('datlab','companies_seattle')

In [128]:
q = list(coll.find())

In [129]:
seattle = pd.DataFrame(q)

In [130]:
seattle = pd.concat([seattle.drop(['location','latitude'], axis=1), seattle['location'].apply(pd.Series)], axis=1)

In [131]:
listacoords = list(seattle['coordinates'])

In [132]:
def count(collection, lista):
    list250 = []
    list500 = []
    for e in lista:
        list250.append(geoQuery(collection,e,250,0))
        list500.append((geoQuery(collection,e,500,250))/2)
    return list250, list500

In [133]:
sb250, sb500 = count(star, listacoords)
air250, air500 = count(air, listacoords)
sch250, sch500 = count(sch, listacoords)
ni250, ni500 = count(night, listacoords)
veg250, veg500 = count(veg, listacoords)
kin250, kin500 = count(kinder, listacoords)

In [134]:
sb250

[1, 0, 0, 5, 1, 0, 1, 5, 1, 0, 5, 1, 0, 1, 0, 0, 3, 3, 0, 0, 0]

In [135]:
facilities = pd.DataFrame(list(zip(sb250,air250,sch250,ni250,veg250,kin250)), columns =["Starbucks","Airports", "Schools","Nightclubs","Veganrests","Kindergarten"])

In [136]:
seattle =pd.concat([seattle, facilities], axis=1)

In [137]:
seattle['250'] = seattle['Starbucks'] + seattle['Airports'] + seattle['Schools'] + seattle['Nightclubs'] + seattle['Veganrests'] + seattle['Kindergarten']

In [138]:
seattle = seattle.loc[(seattle['250']>5)]

In [139]:
coldelete = ['Starbucks','Airports','Schools','Nightclubs','Veganrests','Kindergarten','_id','','description','city','type']
seattle = seattle.drop(coldelete, axis=1)

In [140]:
seattle

,name,category_code,number_of_employees,founded_year,total_money_raised,coordinates,250
3,Implied Solutions,software,1,2008,$0,"[-122.3328401, 47.6042663]",14
6,L4 Mobile,mobile,25,2008,$4.8M,"[-122.2049192, 47.676762]",18
7,Optify,software,45,2008,$10.8M,"[-122.333359, 47.603364]",16
10,Z2Live,games_video,130,2009,$6.5M,"[-122.3399782, 47.6103008]",14
16,GroupieGuide,web,4,2008,$0,"[-122.3374372, 47.6042792]",8
17,Decho,web,150,2008,$0,"[-122.3326145, 47.6024156]",15
